# I need to test this with 2 gamepads...still the keyboard maping is not working as expected

In [7]:
import os
import cv2
import math
import time
import json
import pandas as pd
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta
from dateutil import parser as dtparser

import tkinter as tk
from tkinter import ttk, filedialog, messagebox

from PIL import Image, ImageTk

try:
    import winsound
    _HAS_WINSOUND = True
except Exception:
    winsound = None
    _HAS_WINSOUND = False

try:
    import pygame
    _HAS_PYGAME = True
except Exception:
    pygame = None
    _HAS_PYGAME = False


@dataclass
class Annotation:
    video_path: str
    behavior: str
    kind: str
    start_frame: int
    end_frame: int
    start_time_s: float
    end_time_s: float
    start_datetime: str
    end_datetime: str
    player_id: int
    player_name: str
    animal_id: int


class BehaviourAnnotatorGUI:
    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("Behaviour Annotator")
        self.root.geometry("1250x820")
        self.root.minsize(1100, 720)

        self.cap = None
        self.video_path = ""
        self.fps = 0.0
        self.frame_count = 0
        self.cur_frame_idx = 0
        self.playing = False
        self._last_rendered_idx = None
        self._tk_img = None

        self.start_dt = None

        self.behaviors = []
        self.annotations = []

        self.players = []
        self.player_count = 2
        self.animal_count = 1

        self.active_bouts = {}

        self._binding_player_btn = None
        self._binding_player_key = None
        self._binding_behavior_btn = None
        self._binding_behavior_key = None

        self.sound_enabled = tk.BooleanVar(value=True)
        self.overlay_text = ""
        self.overlay_until = 0.0

        self._last_play_ts = None

        self._known_gamepads = []
        self._pad_owner = {}

        self._build_ui()

        self.root.bind_all("<KeyPress>", self._on_key_press, add="+")
        self.root.after(30, self._ui_tick)

        self._init_pygame()
        self._refresh_gamepads()

        self._ensure_players_animals_ui_synced(force=True)
        self._apply_players_ui_to_state()
        self._refresh_context_dropdowns()
        self._build_bind_rows()
        self._refresh_active_bout_label()
        try:
            self.root.focus_force()
        except Exception:
            pass

    def _build_ui(self):
        self.root.configure(bg="#111316")

        style = ttk.Style()
        try:
            style.theme_use("vista")
        except Exception:
            pass

        style.configure(".", background="#203C49", foreground="#0C4F69", fieldbackground="#2A2E33")
        style.configure("TFrame", background="#111316")
        style.map(
            "TButton",
            background=[("active", "#050505"), ("pressed", "#1A2F3C")],
            foreground=[("active", "#0D660D"), ("pressed", "#EC1717")],
        )
        style.configure("TLabel", background="#111316", foreground="#EAEAEA")
        style.configure("TButton", padding=6)
        style.configure("TLabelframe", background="#111316", foreground="#EAEAEA")
        style.configure("TLabelframe.Label", background="#111316", foreground="#EAEAEA")
        style.configure("TNotebook", background="#111316", borderwidth=0)
        style.configure("TNotebook.Tab", padding=(12, 8))
        style.map("TNotebook.Tab", background=[("selected", "#1A1F24")], foreground=[("selected", "#FFFFFF")])

        style.configure("Ethogram.TButton", padding=3)
        style.configure("Small.TButton", padding=4)

        top = ttk.Frame(self.root)
        top.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        btn_load = ttk.Button(top, text="Load video", command=self.load_video)
        btn_load.pack(side=tk.LEFT)

        self.video_label = ttk.Label(top, text="No video loaded")
        self.video_label.pack(side=tk.LEFT, padx=10)

        ttk.Label(top, text="FPS:").pack(side=tk.LEFT, padx=(20, 4))
        self.fps_var = tk.StringVar(value="")
        self.fps_entry = ttk.Entry(top, width=8, textvariable=self.fps_var)
        self.fps_entry.pack(side=tk.LEFT)
        ttk.Button(top, text="Apply FPS", command=self.apply_fps_override).pack(side=tk.LEFT, padx=6)

        ttk.Label(top, text="Start date/time (optional):").pack(side=tk.LEFT, padx=(20, 4))
        self.startdt_var = tk.StringVar(value="")
        self.startdt_entry = ttk.Entry(top, width=26, textvariable=self.startdt_var)
        self.startdt_entry.pack(side=tk.LEFT)
        ttk.Button(top, text="Set", command=self.apply_start_datetime).pack(side=tk.LEFT, padx=6)

        self.sound_chk = ttk.Checkbutton(top, text="Sounds", variable=self.sound_enabled)
        self.sound_chk.pack(side=tk.LEFT, padx=(20, 0))

        ttk.Button(top, text="Show controls", command=self.show_controls_popup).pack(side=tk.RIGHT)

        main = ttk.Frame(self.root)
        main.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        paned = ttk.Panedwindow(main, orient=tk.HORIZONTAL)
        paned.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        left = ttk.Frame(paned)
        right = ttk.Frame(paned, width=520)
        right.pack_propagate(False)

        paned.add(left, weight=4)
        paned.add(right, weight=1)

        vid_box = ttk.LabelFrame(left, text="Video")
        vid_box.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        self.canvas = tk.Canvas(vid_box, bg="#000000", highlightthickness=0)
        self.canvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        controls = ttk.Frame(vid_box)
        controls.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 8))

        self.btn_play = ttk.Button(controls, text="Play", command=self.toggle_play, state=tk.DISABLED)
        self.btn_play.pack(side=tk.LEFT)

        ttk.Button(
            controls, text="<< -100f", command=lambda: self.step_frames(-100), style="Small.TButton"
        ).pack(side=tk.LEFT, padx=6)
        ttk.Button(controls, text="< -1f", command=lambda: self.step_frames(-1), style="Small.TButton").pack(side=tk.LEFT)
        ttk.Button(controls, text="+1f >", command=lambda: self.step_frames(1), style="Small.TButton").pack(
            side=tk.LEFT, padx=6
        )
        ttk.Button(controls, text="+100f >>", command=lambda: self.step_frames(100), style="Small.TButton").pack(
            side=tk.LEFT
        )

        self.info_var = tk.StringVar(value="Frame: - / - | Time: - s | Timestamp: -")
        ttk.Label(controls, textvariable=self.info_var).pack(side=tk.RIGHT)

        self.slider = ttk.Scale(vid_box, from_=0, to=0, orient=tk.HORIZONTAL, command=self.on_slider)
        self.slider.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 10))

        help_row = ttk.Frame(vid_box)
        help_row.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 10))

        self.quick_help_var = tk.StringVar(value="")
        ttk.Label(help_row, textvariable=self.quick_help_var).pack(side=tk.LEFT, anchor="w")
        self._update_quick_help()

        nb = ttk.Notebook(right)
        nb.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        tab_eth = ttk.Frame(nb)
        tab_multi = ttk.Frame(nb)
        tab_pad = ttk.Frame(nb)
        tab_ann = ttk.Frame(nb)

        nb.add(tab_eth, text="Ethogram")
        nb.add(tab_multi, text="Players/Animals")
        nb.add(tab_pad, text="Inputs")
        nb.add(tab_ann, text="Annotations")

        eth_top = ttk.LabelFrame(tab_eth, text="Ethogram setup")
        eth_top.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        ttk.Label(eth_top, text="Behaviours (comma-separated):").pack(side=tk.TOP, anchor="w")
        self.eth_var = tk.StringVar(value="Replace with your behaviour")
        self.eth_entry = ttk.Entry(eth_top, textvariable=self.eth_var)
        self.eth_entry.pack(side=tk.TOP, fill=tk.X, pady=6)

        ttk.Button(eth_top, text="Apply ethogram", command=self.apply_ethogram).pack(side=tk.TOP, anchor="w")

        mode_row = ttk.Frame(eth_top)
        mode_row.pack(side=tk.TOP, fill=tk.X, pady=(10, 0))
        self.mode_var = tk.StringVar(value="bout")
        ttk.Radiobutton(mode_row, text="Bout mode (Start/End)", value="bout", variable=self.mode_var).pack(side=tk.LEFT)
        ttk.Radiobutton(mode_row, text="Event mode (Mark)", value="event", variable=self.mode_var).pack(
            side=tk.LEFT, padx=12
        )

        policy_row = ttk.Frame(eth_top)
        policy_row.pack(side=tk.TOP, fill=tk.X, pady=(6, 0))
        self.bout_policy_var = tk.StringVar(value="exclusive")
        ttk.Radiobutton(policy_row, text="Exclusive bouts", value="exclusive", variable=self.bout_policy_var).pack(
            side=tk.LEFT
        )
        ttk.Radiobutton(policy_row, text="Parallel bouts", value="parallel", variable=self.bout_policy_var).pack(
            side=tk.LEFT, padx=12
        )

        ctx_row = ttk.Frame(eth_top)
        ctx_row.pack(side=tk.TOP, fill=tk.X, pady=(10, 0))

        ttk.Label(ctx_row, text="GUI Player:").pack(side=tk.LEFT)
        self.gui_player_var = tk.StringVar(value="P1")
        self.gui_player_combo = ttk.Combobox(ctx_row, textvariable=self.gui_player_var, state="readonly", width=6, values=["P1"])
        self.gui_player_combo.pack(side=tk.LEFT, padx=(6, 14))

        ttk.Label(ctx_row, text="GUI Animal:").pack(side=tk.LEFT)
        self.gui_animal_var = tk.StringVar(value="A1")
        self.gui_animal_combo = ttk.Combobox(ctx_row, textvariable=self.gui_animal_var, state="readonly", width=6, values=["A1"])
        self.gui_animal_combo.pack(side=tk.LEFT, padx=(6, 0))

        self.active_bout_var = tk.StringVar(value="Active bouts: none")
        ttk.Label(tab_eth, textvariable=self.active_bout_var).pack(side=tk.TOP, anchor="w", padx=12, pady=(0, 6))

        beh_box = ttk.LabelFrame(tab_eth, text="Ethogram buttons")
        beh_box.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        self.beh_scroll_canvas = tk.Canvas(beh_box, bg="#111316", highlightthickness=0)
        self.beh_scroll_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(10, 0), pady=10)

        beh_scrollbar = ttk.Scrollbar(beh_box, orient=tk.VERTICAL, command=self.beh_scroll_canvas.yview)
        beh_scrollbar.pack(side=tk.RIGHT, fill=tk.Y, padx=(0, 10), pady=10)

        self.beh_scroll_canvas.configure(yscrollcommand=beh_scrollbar.set)

        self.beh_btn_frame = ttk.Frame(self.beh_scroll_canvas)
        self.beh_btn_window = self.beh_scroll_canvas.create_window((0, 0), window=self.beh_btn_frame, anchor="nw")

        self.beh_btn_frame.bind("<Configure>", self._on_beh_frame_configure)
        self.beh_scroll_canvas.bind("<Configure>", self._on_beh_canvas_configure)
        self.beh_scroll_canvas.bind_all("<MouseWheel>", self._on_mousewheel)

        mp_top = ttk.LabelFrame(tab_multi, text="Multiplayer / Multi-animal")
        mp_top.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        row0 = ttk.Frame(mp_top)
        row0.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(10, 6))

        ttk.Label(row0, text="Players:").pack(side=tk.LEFT)
        self.player_count_var = tk.IntVar(value=2)
        self.player_count_spin = ttk.Spinbox(row0, from_=1, to=6, width=6, textvariable=self.player_count_var, command=self._rebuild_players_ui)
        self.player_count_spin.pack(side=tk.LEFT, padx=(6, 14))
        self.player_count_spin.bind("<KeyRelease>", lambda _e: self._ensure_players_animals_ui_synced(force=True))

        ttk.Label(row0, text="Animals:").pack(side=tk.LEFT)
        self.animal_count_var = tk.IntVar(value=1)
        self.animal_count_spin = ttk.Spinbox(row0, from_=1, to=10, width=6, textvariable=self.animal_count_var, command=self._rebuild_players_ui)
        self.animal_count_spin.pack(side=tk.LEFT, padx=(6, 14))
        self.animal_count_spin.bind("<KeyRelease>", lambda _e: self._ensure_players_animals_ui_synced(force=True))

        ttk.Button(row0, text="Apply players/animals", command=self.apply_players_animals).pack(side=tk.LEFT, padx=(6, 0))
        ttk.Button(row0, text="Auto-split behaviours", command=self.auto_split_behaviours).pack(side=tk.RIGHT)

        self.players_frame = ttk.Frame(mp_top)
        self.players_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        pad_box = ttk.LabelFrame(tab_pad, text="Gamepads")
        pad_box.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        self.pad_status_var = tk.StringVar(value="Gamepad: not initialised")
        ttk.Label(pad_box, textvariable=self.pad_status_var).pack(side=tk.TOP, anchor="w", padx=10, pady=(8, 6))

        pad_row = ttk.Frame(pad_box)
        pad_row.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 6))

        self.pad_list_var = tk.StringVar(value="")
        self.pad_combo = ttk.Combobox(pad_row, textvariable=self.pad_list_var, state="readonly", width=32, values=[])
        self.pad_combo.pack(side=tk.LEFT, fill=tk.X, expand=True)

        ttk.Button(pad_row, text="Refresh", command=self._refresh_gamepads).pack(side=tk.LEFT, padx=6)

        bind_head = ttk.LabelFrame(tab_pad, text="Binding context")
        bind_head.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 10))

        r = ttk.Frame(bind_head)
        r.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        ttk.Label(r, text="Bind for player:").pack(side=tk.LEFT)
        self.bind_player_var = tk.StringVar(value="P1")
        self.bind_player_combo = ttk.Combobox(r, textvariable=self.bind_player_var, state="readonly", width=6, values=["P1"])
        self.bind_player_combo.pack(side=tk.LEFT, padx=(6, 14))
        self.bind_player_combo.bind("<<ComboboxSelected>>", lambda _e: self._build_bind_rows())

        ttk.Label(r, text="Keyboard targets player:").pack(side=tk.LEFT)
        self.kb_target_player_var = tk.StringVar(value="P1")
        self.kb_target_player_combo = ttk.Combobox(r, textvariable=self.kb_target_player_var, state="readonly", width=6, values=["P1"])
        self.kb_target_player_combo.pack(side=tk.LEFT, padx=(6, 14))

        ttk.Button(r, text="Save mapping", command=self.save_mapping).pack(side=tk.LEFT)
        ttk.Button(r, text="Load mapping", command=self.load_mapping).pack(side=tk.LEFT, padx=6)
        ttk.Button(r, text="Clear mapping", command=self.clear_mapping).pack(side=tk.LEFT, padx=6)
        ttk.Button(r, text="Show mapping", command=self.show_controls_popup).pack(side=tk.RIGHT)

        bind_box = ttk.LabelFrame(tab_pad, text="Bind behaviour keys/buttons (per player)")
        bind_box.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

        self.bind_frame = ttk.Frame(bind_box)
        self.bind_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.bind_note_var = tk.StringVar(value="")
        ttk.Label(tab_pad, textvariable=self.bind_note_var).pack(side=tk.TOP, anchor="w", padx=12, pady=(0, 10))

        ann_box = ttk.LabelFrame(tab_ann, text="Annotations")
        ann_box.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        cols = ("player", "animal", "behavior", "kind", "start_frame", "end_frame", "start_time_s", "end_time_s", "start_datetime", "end_datetime")
        self.tree = ttk.Treeview(ann_box, columns=cols, show="headings", height=12)
        for c in cols:
            self.tree.heading(c, text=c)
            if c in ("player", "animal"):
                self.tree.column(c, width=70, anchor="center")
            elif c in ("behavior", "kind"):
                self.tree.column(c, width=120, anchor="center")
            else:
                self.tree.column(c, width=135, anchor="center")
        self.tree.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

        btn_row = ttk.Frame(ann_box)
        btn_row.pack(side=tk.TOP, fill=tk.X, padx=10, pady=(0, 10))

        ttk.Button(btn_row, text="Undo last", command=self.undo_last).pack(side=tk.LEFT)
        ttk.Button(btn_row, text="Delete selected", command=self.delete_selected).pack(side=tk.LEFT, padx=6)
        ttk.Button(btn_row, text="Clear all", command=self.clear_all).pack(side=tk.LEFT, padx=6)
        ttk.Button(btn_row, text="Save CSV", command=self.save_csv).pack(side=tk.RIGHT)

        footer = ttk.Frame(self.root)
        footer.pack(side=tk.BOTTOM, fill=tk.X)
        ttk.Label(
            footer,
            text="© 2026 BiRBSLAB | UiT | Norway",
            font=("Courier New", 10),
            foreground="orange",
        ).pack(pady=(2, 0))

        self.hyperlink_label = tk.Label(
            footer,
            text="Developed by Hamid Taghipourbibalan",
            font=("Courier New", 8),
            foreground="#1384e7",
            bg="#111316",
            cursor="hand2",
        )
        self.hyperlink_label.pack(pady=(0, 6))
        self.hyperlink_label.bind(
            "<Button-1>",
            lambda e: self.open_hyperlink("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"),
        )

        self._rebuild_players_ui()
        self.apply_ethogram()
        self.apply_players_animals()

    def _ensure_players_animals_ui_synced(self, force: bool = False):
        try:
            pc = int(self.player_count_var.get() or 1)
        except Exception:
            pc = 1
        try:
            ac = int(self.animal_count_var.get() or 1)
        except Exception:
            ac = 1
        pc = max(1, min(6, pc))
        ac = max(1, min(10, ac))

        need = force or (pc != int(self.player_count)) or (ac != int(self.animal_count)) or (not hasattr(self, "player_ui")) or (len(self.player_ui) != pc)
        if not need:
            return

        prev_rows = []
        if hasattr(self, "player_ui"):
            for ui in self.player_ui:
                prev_rows.append(
                    {
                        "name": ui["name_var"].get(),
                        "input": ui["input_var"].get(),
                        "gp": ui["gp_var"].get(),
                        "animal": ui["animal_var"].get(),
                        "allowed": ui["allowed_var"].get(),
                    }
                )

        self.player_count_var.set(pc)
        self.animal_count_var.set(ac)
        self._rebuild_players_ui()

        for i in range(min(len(prev_rows), len(self.player_ui))):
            ui = self.player_ui[i]
            pr = prev_rows[i]
            ui["name_var"].set(pr.get("name", ui["name_var"].get()))
            ui["input_var"].set(pr.get("input", ui["input_var"].get()))
            ui["gp_var"].set(pr.get("gp", ui["gp_var"].get()))
            ui["animal_var"].set(pr.get("animal", ui["animal_var"].get()))
            ui["allowed_var"].set(pr.get("allowed", ui["allowed_var"].get()))
            self._sync_player_row_widgets(i)

    def apply_fps_override(self):
        s = (self.fps_var.get() or "").strip()
        if not s:
            if self.cap is not None:
                try:
                    fps = float(self.cap.get(cv2.CAP_PROP_FPS) or 0.0)
                except Exception:
                    fps = 0.0
                if fps is None or fps <= 0 or math.isnan(fps):
                    fps = 0.0
                self.fps = float(fps)
                self.fps_var.set("" if self.fps <= 0 else f"{self.fps:.3f}")
                self._set_overlay("FPS RESET", 900)
                self._play_sound("event")
                self._last_rendered_idx = None
                self.render_frame(self.cur_frame_idx)
            else:
                self.fps = 0.0
                self._set_overlay("FPS CLEARED", 900)
                self._play_sound("event")
            return

        try:
            fps = float(s)
        except Exception:
            messagebox.showwarning("FPS", "Invalid FPS value.")
            self._play_sound("warn")
            return

        if fps <= 0 or math.isnan(fps) or math.isinf(fps):
            messagebox.showwarning("FPS", "FPS must be > 0.")
            self._play_sound("warn")
            return

        self.fps = float(fps)
        self.fps_var.set(f"{self.fps:.3f}")
        self._set_overlay("FPS APPLIED", 900)
        self._play_sound("event")
        self._last_rendered_idx = None
        self.render_frame(self.cur_frame_idx)

    def apply_start_datetime(self):
        raw = (self.startdt_var.get() or "").strip()
        if not raw:
            self.start_dt = None
            self._set_overlay("START DT CLEARED", 900)
            self._play_sound("event")
            self._last_rendered_idx = None
            self.render_frame(self.cur_frame_idx)
            return
        try:
            dt = dtparser.parse(raw)
        except Exception:
            messagebox.showwarning("Start datetime", "Could not parse start date/time.")
            self._play_sound("warn")
            return
        self.start_dt = dt
        self._set_overlay("START DT SET", 900)
        self._play_sound("event")
        self._last_rendered_idx = None
        self.render_frame(self.cur_frame_idx)

    def _rebuild_players_ui(self):
        for w in self.players_frame.winfo_children():
            w.destroy()

        try:
            pc = int(self.player_count_var.get() or 1)
        except Exception:
            pc = 1
        try:
            ac = int(self.animal_count_var.get() or 1)
        except Exception:
            ac = 1
        if pc < 1:
            pc = 1
        if ac < 1:
            ac = 1
        self.player_count = pc
        self.animal_count = ac

        hdr = ttk.Frame(self.players_frame)
        hdr.pack(side=tk.TOP, fill=tk.X, pady=(0, 4))
        ttk.Label(hdr, text="Player").grid(row=0, column=0, sticky="w")
        ttk.Label(hdr, text="Name").grid(row=0, column=1, sticky="w", padx=(6, 0))
        ttk.Label(hdr, text="Input").grid(row=0, column=2, sticky="w", padx=(6, 0))
        ttk.Label(hdr, text="Gamepad").grid(row=0, column=3, sticky="w", padx=(6, 0))
        ttk.Label(hdr, text="Animal").grid(row=0, column=4, sticky="w", padx=(6, 0))
        ttk.Label(hdr, text="Allowed behaviours (*=all or comma list)").grid(row=0, column=5, sticky="w", padx=(6, 0))

        self.player_ui = []

        try:
            gamepads = list(self.pad_combo.cget("values") or [])
        except Exception:
            gamepads = []

        animals = [f"A{i+1}" for i in range(self.animal_count)] + ["ALL"]

        for i in range(self.player_count):
            row = ttk.Frame(self.players_frame)
            row.pack(side=tk.TOP, fill=tk.X, pady=3)

            ttk.Label(row, text=f"P{i+1}", width=4).grid(row=0, column=0, sticky="w")

            name_var = tk.StringVar(value=f"Player {i+1}")
            ttk.Entry(row, textvariable=name_var, width=14).grid(row=0, column=1, sticky="w", padx=(6, 0))

            input_var = tk.StringVar(value="Keyboard" if i == 0 else "Gamepad")
            input_combo = ttk.Combobox(row, textvariable=input_var, state="readonly", width=10, values=["Keyboard", "Gamepad"])
            input_combo.grid(row=0, column=2, sticky="w", padx=(6, 0))

            gp_var = tk.StringVar(value="")
            gp_combo = ttk.Combobox(row, textvariable=gp_var, state="readonly", width=28, values=gamepads)
            gp_combo.grid(row=0, column=3, sticky="we", padx=(6, 0))

            animal_var = tk.StringVar(value=f"A{min(i+1, self.animal_count)}")
            ttk.Combobox(row, textvariable=animal_var, state="readonly", width=6, values=animals).grid(
                row=0, column=4, sticky="w", padx=(6, 0)
            )

            allowed_var = tk.StringVar(value="*")
            ttk.Entry(row, textvariable=allowed_var, width=34).grid(row=0, column=5, sticky="we", padx=(6, 0))

            row.grid_columnconfigure(3, weight=1)
            row.grid_columnconfigure(5, weight=2)

            self.player_ui.append(
                {
                    "name_var": name_var,
                    "input_var": input_var,
                    "gp_var": gp_var,
                    "animal_var": animal_var,
                    "allowed_var": allowed_var,
                    "gp_combo": gp_combo,
                    "input_combo": input_combo,
                }
            )

            def _mk_on_input_change(ix=i):
                return lambda _e=None: self._sync_player_row_widgets(ix)

            input_combo.bind("<<ComboboxSelected>>", _mk_on_input_change(i))

        for i in range(self.player_count):
            self._sync_player_row_widgets(i)

    def _sync_player_row_widgets(self, i: int):
        if not hasattr(self, "player_ui") or i < 0 or i >= len(self.player_ui):
            return
        ui = self.player_ui[i]
        input_type = (ui["input_var"].get() or "").strip()
        if input_type == "Gamepad":
            ui["gp_combo"].configure(state="readonly")
            if not (ui["gp_var"].get() or "").strip():
                items = list(ui["gp_combo"].cget("values") or [])
                used = set()
                for j, u2 in enumerate(self.player_ui):
                    if j == i:
                        continue
                    if (u2["input_var"].get() or "").strip() != "Gamepad":
                        continue
                    s = (u2["gp_var"].get() or "").strip()
                    if s:
                        used.add(s)
                pick = ""
                for s in items:
                    if s not in used:
                        pick = s
                        break
                if not pick and items:
                    pick = items[0]
                ui["gp_var"].set(pick)
        else:
            ui["gp_var"].set("")
            ui["gp_combo"].configure(state="disabled")

    def apply_players_animals(self):
        self._ensure_players_animals_ui_synced(force=True)
        self._apply_players_ui_to_state()
        self._refresh_context_dropdowns()
        self._build_bind_rows()
        self._refresh_active_bout_label()
        self._set_overlay("PLAYERS/APPLIED", 900)
        self._play_sound("event")
        try:
            self.root.focus_force()
        except Exception:
            pass

    def _apply_players_ui_to_state(self):
        self._ensure_players_animals_ui_synced(force=False)

        old_players = {p["id"]: p for p in self.players}
        new_players = []

        self._pad_owner = {}
        requested_gamepads = []

        for i in range(self.player_count):
            ui = self.player_ui[i]
            pid = i + 1
            name = ui["name_var"].get().strip() or f"Player {pid}"
            input_type = ui["input_var"].get().strip() or "Keyboard"
            gp_sel = (ui["gp_var"].get() or "").strip()
            animal_sel = ui["animal_var"].get().strip() or "A1"
            allowed_raw = ui["allowed_var"].get().strip()

            animal_id = 1
            if animal_sel == "ALL":
                animal_id = 0
            else:
                try:
                    animal_id = int(animal_sel.replace("A", ""))
                except Exception:
                    animal_id = 1
                if animal_id < 1:
                    animal_id = 1
                if animal_id > self.animal_count:
                    animal_id = self.animal_count

            allowed_set = None
            if allowed_raw and allowed_raw != "*":
                allowed = [x.strip() for x in allowed_raw.split(",") if x.strip()]
                allowed_set = set(allowed) if allowed else None

            prev = old_players.get(pid, None)
            if prev is None:
                prev = {
                    "id": pid,
                    "name": name,
                    "input": input_type,
                    "pad_idx": None,
                    "pad_obj": None,
                    "pad_name": "",
                    "pad_guid": "",
                    "enabled": False,
                    "animal_id": animal_id,
                    "allowed_behaviors": allowed_set,
                    "behavior_to_button": {},
                    "button_to_behavior": {},
                    "behavior_to_key": {},
                    "key_to_behavior": {},
                    "pad_last_btn": {},
                    "pad_last_hat": (0, 0),
                    "pad_repeat_t": 0.0,
                    "pad_btn_state": None,
                }

            prev["id"] = pid
            prev["name"] = name
            prev["input"] = input_type
            prev["animal_id"] = animal_id
            prev["allowed_behaviors"] = allowed_set

            if input_type == "Gamepad":
                idx = None
                if gp_sel:
                    try:
                        idx = int(gp_sel.split(":")[0].strip())
                    except Exception:
                        idx = None
                requested_gamepads.append((pid, idx))
                prev["pad_idx"] = idx
            else:
                prev["pad_idx"] = None
                prev["pad_obj"] = None
                prev["enabled"] = False
                prev["pad_name"] = ""
                prev["pad_guid"] = ""
                prev["pad_last_btn"] = {}
                prev["pad_last_hat"] = (0, 0)
                prev["pad_repeat_t"] = 0.0
                prev["pad_btn_state"] = None

            new_players.append(prev)

        used = set()
        duplicates = []
        for pid, idx in requested_gamepads:
            if idx is None or idx < 0:
                continue
            if idx in used:
                duplicates.append((pid, idx))
            else:
                used.add(idx)

        if duplicates:
            msg_parts = []
            for pid, idx in duplicates:
                msg_parts.append(f"P{pid} (pad {idx})")
                p = None
                for pp in new_players:
                    if int(pp.get("id", 0)) == int(pid):
                        p = pp
                        break
                if p is not None:
                    p["pad_idx"] = None
                    p["pad_obj"] = None
                    p["enabled"] = False
                    p["pad_name"] = ""
                    p["pad_guid"] = ""
                    p["pad_last_btn"] = {}
                    p["pad_last_hat"] = (0, 0)
                    p["pad_repeat_t"] = 0.0
                    p["pad_btn_state"] = None
            self._set_overlay("DUP PAD -> DISABLED: " + ", ".join(msg_parts), 1400)
            self._play_sound("warn")

        for p in new_players:
            if p.get("input") == "Gamepad":
                idx = p.get("pad_idx", None)
                if idx is None or idx < 0:
                    p["pad_obj"] = None
                    p["enabled"] = False
                    p["pad_name"] = ""
                    p["pad_guid"] = ""
                    p["pad_last_btn"] = {}
                    p["pad_last_hat"] = (0, 0)
                    p["pad_repeat_t"] = 0.0
                    p["pad_btn_state"] = None
                else:
                    self._connect_player_gamepad(p, int(idx))
                    if p.get("enabled"):
                        self._pad_owner[int(idx)] = int(p["id"])

        self.players = new_players

    def auto_split_behaviours(self):
        if not self.behaviors:
            self.apply_ethogram()
        if not self.behaviors:
            return
        try:
            pc = int(self.player_count_var.get() or 1)
        except Exception:
            pc = 1
        if pc < 1:
            pc = 1
        parts = [[] for _ in range(pc)]
        for i, b in enumerate(self.behaviors):
            parts[i % pc].append(b)
        for i in range(min(pc, len(self.player_ui))):
            self.player_ui[i]["allowed_var"].set(", ".join(parts[i]) if parts[i] else "*")
        self._set_overlay("AUTO-SPLIT", 900)
        self._play_sound("event")

    def _refresh_context_dropdowns(self):
        pvals = [f"P{i+1}" for i in range(self.player_count)]
        avals = [f"A{i+1}" for i in range(self.animal_count)]
        self.gui_player_combo.configure(values=pvals)
        self.bind_player_combo.configure(values=pvals)
        self.kb_target_player_combo.configure(values=pvals)

        if self.gui_player_var.get() not in pvals:
            self.gui_player_var.set(pvals[0] if pvals else "P1")
        if self.bind_player_var.get() not in pvals:
            self.bind_player_var.set(pvals[0] if pvals else "P1")
        if self.kb_target_player_var.get() not in pvals:
            self.kb_target_player_var.set(pvals[0] if pvals else "P1")

        self.gui_animal_combo.configure(values=avals)
        if self.gui_animal_var.get() not in avals:
            self.gui_animal_var.set(avals[0] if avals else "A1")

    def _set_overlay(self, text: str, duration_ms: int = 1200):
        self.overlay_text = text
        self.overlay_until = time.time() + max(0.05, duration_ms / 1000.0)
        if self.cap is not None:
            self._last_rendered_idx = None
            self.render_frame(self.cur_frame_idx)

    def _play_sound(self, kind: str):
        if not bool(self.sound_enabled.get()):
            return
        if _HAS_WINSOUND:
            try:
                if kind == "start":
                    winsound.Beep(880, 90)
                    winsound.Beep(1320, 70)
                elif kind == "end":
                    winsound.Beep(1320, 70)
                    winsound.Beep(880, 90)
                elif kind == "event":
                    winsound.Beep(1046, 70)
                elif kind == "undo":
                    winsound.Beep(740, 80)
                elif kind == "warn":
                    winsound.Beep(330, 120)
                else:
                    winsound.Beep(880, 60)
            except Exception:
                try:
                    self.root.bell()
                except Exception:
                    pass
        else:
            try:
                self.root.bell()
            except Exception:
                pass

    def _on_mousewheel(self, e):
        try:
            if self.beh_scroll_canvas.winfo_ismapped():
                self.beh_scroll_canvas.yview_scroll(int(-1 * (e.delta / 120)), "units")
        except Exception:
            pass

    def _on_beh_frame_configure(self, _e=None):
        try:
            self.beh_scroll_canvas.configure(scrollregion=self.beh_scroll_canvas.bbox("all"))
        except Exception:
            pass

    def _on_beh_canvas_configure(self, _e=None):
        try:
            self.beh_scroll_canvas.itemconfig(self.beh_btn_window, width=self.beh_scroll_canvas.winfo_width())
            self._layout_ethogram_buttons()
        except Exception:
            pass

    def _update_quick_help(self):
        self.quick_help_var.set(
            "Keyboard: Space/Enter=Play/Pause | ←/→=±1 frame | ↑/↓=±100 frames | Gamepad: D-pad=±1/±100 | Start=Play/Pause | Back=Undo"
        )

    def _init_pygame(self):
        if not _HAS_PYGAME:
            self.pad_status_var.set("Gamepad: pygame not installed")
            return
        try:
            pygame.init()
            pygame.joystick.init()
            self.pad_status_var.set("Gamepad: ready")
        except Exception:
            self.pad_status_var.set("Gamepad: init failed")

    def _refresh_gamepads(self):
        self._known_gamepads = []
        if not _HAS_PYGAME:
            self.pad_combo.configure(values=[])
            self.pad_list_var.set("")
            self.pad_status_var.set("Gamepad: pygame not installed")
            return
        try:
            pygame.joystick.quit()
            pygame.joystick.init()
            n = pygame.joystick.get_count()
            items = []
            for i in range(n):
                js = pygame.joystick.Joystick(i)
                js.init()
                name = js.get_name()
                guid = ""
                try:
                    guid = js.get_guid()
                except Exception:
                    guid = ""
                items.append(f"{i}: {name} {('[' + guid + ']') if guid else ''}".strip())
                self._known_gamepads.append({"idx": i, "name": name, "guid": guid})
            self.pad_combo.configure(values=items)
            if items:
                self.pad_list_var.set(items[0])
            else:
                self.pad_list_var.set("")
            self.pad_status_var.set(f"Gamepad: {n} detected")
            if hasattr(self, "player_ui"):
                for ui in self.player_ui:
                    ui["gp_combo"].configure(values=items)
                for i in range(len(self.player_ui)):
                    self._sync_player_row_widgets(i)
        except Exception:
            self.pad_combo.configure(values=[])
            self.pad_list_var.set("")
            self.pad_status_var.set("Gamepad: refresh failed")

    def _connect_player_gamepad(self, player: dict, idx: int):
        if not _HAS_PYGAME:
            player["pad_obj"] = None
            player["enabled"] = False
            player["pad_name"] = ""
            player["pad_guid"] = ""
            player["pad_last_btn"] = {}
            player["pad_last_hat"] = (0, 0)
            player["pad_repeat_t"] = 0.0
            player["pad_btn_state"] = None
            return
        try:
            js = pygame.joystick.Joystick(idx)
            js.init()
            player["pad_obj"] = js
            player["enabled"] = True
            player["pad_last_btn"] = {}
            player["pad_last_hat"] = (0, 0)
            player["pad_repeat_t"] = 0.0
            player["pad_name"] = js.get_name()
            try:
                player["pad_guid"] = js.get_guid()
            except Exception:
                player["pad_guid"] = ""
            try:
                nb = js.get_numbuttons()
            except Exception:
                nb = 0
            player["pad_btn_state"] = [0] * max(0, int(nb))
        except Exception:
            player["pad_obj"] = None
            player["enabled"] = False
            player["pad_name"] = ""
            player["pad_guid"] = ""
            player["pad_last_btn"] = {}
            player["pad_last_hat"] = (0, 0)
            player["pad_repeat_t"] = 0.0
            player["pad_btn_state"] = None

    def _get_player_by_id(self, pid: int):
        for p in self.players:
            if p.get("id") == pid:
                return p
        return None

    def _selected_gui_player_id(self) -> int:
        s = self.gui_player_var.get().strip()
        try:
            return int(s.replace("P", ""))
        except Exception:
            return 1

    def _selected_gui_animal_id(self) -> int:
        s = self.gui_animal_var.get().strip()
        try:
            return int(s.replace("A", ""))
        except Exception:
            return 1

    def _selected_bind_player_id(self) -> int:
        s = self.bind_player_var.get().strip()
        try:
            return int(s.replace("P", ""))
        except Exception:
            return 1

    def _selected_kb_target_player_id(self) -> int:
        s = self.kb_target_player_var.get().strip()
        try:
            return int(s.replace("P", ""))
        except Exception:
            return 1

    def _build_bind_rows(self):
        for w in self.bind_frame.winfo_children():
            w.destroy()

        pid = self._selected_bind_player_id()
        p = self._get_player_by_id(pid)
        if p is None:
            return

        header = ttk.Frame(self.bind_frame)
        header.pack(side=tk.TOP, fill=tk.X, pady=(0, 4))
        ttk.Label(header, text="Behaviour").pack(side=tk.LEFT)
        ttk.Label(header, text="Btn").pack(side=tk.LEFT, padx=(80, 0))
        ttk.Label(header, text="Key").pack(side=tk.LEFT, padx=(40, 0))

        for b in self.behaviors:
            row = ttk.Frame(self.bind_frame)
            row.pack(side=tk.TOP, fill=tk.X, pady=2)

            ttk.Label(row, text=b, width=18).pack(side=tk.LEFT)

            btn_txt = tk.StringVar(value=self._fmt_btn_mapping(p, b))
            key_txt = tk.StringVar(value=self._fmt_key_mapping(p, b))

            ttk.Label(row, textvariable=btn_txt, width=10).pack(side=tk.LEFT, padx=(10, 0))

            ttk.Button(row, text="Bind Btn", command=lambda bb=b: self.start_bind_button(pid, bb), style="Small.TButton").pack(side=tk.LEFT, padx=6)
            ttk.Button(row, text="Clear Btn", command=lambda bb=b: self.clear_bind_button(pid, bb), style="Small.TButton").pack(side=tk.LEFT, padx=2)

            ttk.Label(row, textvariable=key_txt, width=12).pack(side=tk.LEFT, padx=(10, 0))

            ttk.Button(row, text="Bind Key", command=lambda bb=b: self.start_bind_key(pid, bb), style="Small.TButton").pack(side=tk.LEFT, padx=6)
            ttk.Button(row, text="Clear Key", command=lambda bb=b: self.clear_bind_key(pid, bb), style="Small.TButton").pack(side=tk.LEFT, padx=2)

            row._btn_txt = btn_txt
            row._key_txt = key_txt
            row._beh = b

    def _refresh_bind_labels(self):
        pid = self._selected_bind_player_id()
        p = self._get_player_by_id(pid)
        if p is None:
            return
        for w in self.bind_frame.winfo_children():
            if hasattr(w, "_btn_txt") and hasattr(w, "_beh"):
                b = w._beh
                w._btn_txt.set(self._fmt_btn_mapping(p, b))
                w._key_txt.set(self._fmt_key_mapping(p, b))

    def _fmt_btn_mapping(self, player: dict, behavior: str) -> str:
        v = player["behavior_to_button"].get(behavior, None)
        return "-" if v is None else str(v)

    def _fmt_key_mapping(self, player: dict, behavior: str) -> str:
        v = player["behavior_to_key"].get(behavior, "")
        return "-" if not v else v

    def start_bind_button(self, player_id: int, behavior: str):
        p = self._get_player_by_id(player_id)
        if p is None:
            return
        if p.get("input") != "Gamepad" or not p.get("enabled") or p.get("pad_obj") is None:
            messagebox.showwarning("Gamepad", "This player is not connected to a gamepad.")
            return
        self._binding_player_btn = player_id
        self._binding_behavior_btn = behavior
        self._binding_player_key = None
        self._binding_behavior_key = None
        self.bind_note_var.set(f"Press a gamepad button to bind: {behavior} (P{player_id})")
        try:
            self.root.focus_force()
        except Exception:
            pass

    def start_bind_key(self, player_id: int, behavior: str):
        self._binding_player_key = player_id
        self._binding_behavior_key = behavior
        self._binding_player_btn = None
        self._binding_behavior_btn = None
        self.bind_note_var.set(f"Press a keyboard key to bind: {behavior} (P{player_id})")
        try:
            self.root.focus_force()
        except Exception:
            pass

    def clear_bind_button(self, player_id: int, behavior: str):
        p = self._get_player_by_id(player_id)
        if p is None:
            return
        old_btn = p["behavior_to_button"].pop(behavior, None)
        if old_btn is not None:
            p["button_to_behavior"].pop(old_btn, None)
        self._refresh_bind_labels()

    def clear_bind_key(self, player_id: int, behavior: str):
        p = self._get_player_by_id(player_id)
        if p is None:
            return
        old_key = p["behavior_to_key"].pop(behavior, "")
        if old_key:
            p["key_to_behavior"].pop(old_key, None)
        self._refresh_bind_labels()

    def _assign_button(self, player: dict, behavior: str, btn_idx: int):
        old_btn = player["behavior_to_button"].get(behavior, None)
        if old_btn is not None:
            player["button_to_behavior"].pop(old_btn, None)

        old_beh = player["button_to_behavior"].get(btn_idx, None)
        if old_beh is not None:
            player["behavior_to_button"].pop(old_beh, None)

        player["behavior_to_button"][behavior] = btn_idx
        player["button_to_behavior"][btn_idx] = behavior
        self._refresh_bind_labels()

    def _assign_key(self, player: dict, behavior: str, key_sym: str):
        old_key = player["behavior_to_key"].get(behavior, "")
        if old_key:
            player["key_to_behavior"].pop(old_key, None)

        old_beh = player["key_to_behavior"].get(key_sym, None)
        if old_beh is not None:
            player["behavior_to_key"].pop(old_beh, None)

        player["behavior_to_key"][behavior] = key_sym
        player["key_to_behavior"][key_sym] = behavior
        self._refresh_bind_labels()

    def save_mapping(self):
        if not self.players:
            messagebox.showwarning("Mapping", "No players configured.")
            return
        if not self.behaviors:
            messagebox.showwarning("Mapping", "Define behaviours first.")
            return
        out_path = filedialog.asksaveasfilename(
            title="Save mapping",
            defaultextension=".json",
            initialfile="multiplayer_mapping.json",
            filetypes=[("JSON", "*.json")],
        )
        if not out_path:
            return

        players_data = []
        for p in self.players:
            players_data.append(
                {
                    "id": int(p["id"]),
                    "name": str(p["name"]),
                    "input": str(p["input"]),
                    "pad_idx": None if p.get("pad_idx") is None else int(p["pad_idx"]),
                    "pad_name": str(p.get("pad_name", "")),
                    "pad_guid": str(p.get("pad_guid", "")),
                    "animal_id": int(p.get("animal_id", 1)),
                    "allowed_behaviors": None if p.get("allowed_behaviors") is None else sorted(list(p.get("allowed_behaviors"))),
                    "behavior_to_button": {k: int(v) for k, v in (p.get("behavior_to_button") or {}).items() if v is not None},
                    "behavior_to_key": {k: str(v) for k, v in (p.get("behavior_to_key") or {}).items() if v},
                }
            )

        data = {
            "version": 1,
            "players": players_data,
            "behaviors": list(self.behaviors),
            "player_count": int(self.player_count),
            "animal_count": int(self.animal_count),
        }

        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)

        messagebox.showinfo("Saved", f"Saved:\n{out_path}")
        self._set_overlay("MAPPING SAVED", 900)
        self._play_sound("event")

    def load_mapping(self):
        path = filedialog.askopenfilename(
            title="Load mapping",
            filetypes=[("JSON", "*.json")],
        )
        if not path:
            return
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception:
            messagebox.showerror("Mapping", "Could not read mapping JSON.")
            return

        pc = int(data.get("player_count") or 1)
        ac = int(data.get("animal_count") or 1)
        if pc < 1:
            pc = 1
        if ac < 1:
            ac = 1

        self.player_count_var.set(pc)
        self.animal_count_var.set(ac)
        self._ensure_players_animals_ui_synced(force=True)

        players_data = data.get("players") or []
        items = list(self.pad_combo.cget("values") or [])

        for i in range(min(len(players_data), len(self.player_ui))):
            p = players_data[i]
            ui = self.player_ui[i]
            ui["name_var"].set(str(p.get("name") or f"Player {i+1}"))
            ui["input_var"].set(str(p.get("input") or ("Keyboard" if i == 0 else "Gamepad")))
            animal_id = int(p.get("animal_id") or 1)
            if animal_id == 0:
                ui["animal_var"].set("ALL")
            else:
                ui["animal_var"].set(f"A{animal_id}")
            ab = p.get("allowed_behaviors", None)
            if ab is None:
                ui["allowed_var"].set("*")
            else:
                ui["allowed_var"].set(", ".join([str(x) for x in ab]) if ab else "*")

            pad_idx = p.get("pad_idx", None)
            if ui["input_var"].get().strip() != "Gamepad":
                ui["gp_var"].set("")
            else:
                if pad_idx is None:
                    ui["gp_var"].set(items[0] if items else "")
                else:
                    found = ""
                    for s in items:
                        try:
                            if int(s.split(":")[0].strip()) == int(pad_idx):
                                found = s
                                break
                        except Exception:
                            pass
                    ui["gp_var"].set(found if found else (items[0] if items else ""))

        for i in range(len(self.player_ui)):
            self._sync_player_row_widgets(i)

        self._apply_players_ui_to_state()

        for pdata in players_data:
            pid = int(pdata.get("id") or 0)
            p = self._get_player_by_id(pid)
            if p is None:
                continue
            b2b = pdata.get("behavior_to_button", {}) or {}
            b2k = pdata.get("behavior_to_key", {}) or {}

            p["behavior_to_button"].clear()
            p["button_to_behavior"].clear()
            p["behavior_to_key"].clear()
            p["key_to_behavior"].clear()

            for beh, btn in b2b.items():
                try:
                    btn_i = int(btn)
                except Exception:
                    continue
                beh_s = str(beh)
                p["behavior_to_button"][beh_s] = btn_i
                p["button_to_behavior"][btn_i] = beh_s

            for beh, key in b2k.items():
                key_s = str(key)
                if not key_s:
                    continue
                beh_s = str(beh)
                p["behavior_to_key"][beh_s] = key_s
                p["key_to_behavior"][key_s] = beh_s

        self._refresh_context_dropdowns()
        self._build_bind_rows()
        self._refresh_active_bout_label()
        self.bind_note_var.set("Mapping loaded")
        self._set_overlay("MAPPING LOADED", 900)
        self._play_sound("event")
        try:
            self.root.focus_force()
        except Exception:
            pass

    def clear_mapping(self):
        for p in self.players:
            p["behavior_to_button"].clear()
            p["button_to_behavior"].clear()
            p["behavior_to_key"].clear()
            p["key_to_behavior"].clear()
        self._refresh_bind_labels()
        self.bind_note_var.set("Mapping cleared")
        self._set_overlay("MAPPING CLEARED", 900)
        self._play_sound("event")
        try:
            self.root.focus_force()
        except Exception:
            pass

    def _on_key_press(self, e: tk.Event):
        if self._binding_player_key is not None and self._binding_behavior_key is not None:
            pid = int(self._binding_player_key)
            beh = str(self._binding_behavior_key)
            self._binding_player_key = None
            self._binding_behavior_key = None
            self._binding_player_btn = None
            self._binding_behavior_btn = None
            p = self._get_player_by_id(pid)
            if p is not None:
                key_sym = e.keysym
                self._assign_key(p, beh, key_sym)
                self.bind_note_var.set(f"Bound key {key_sym} to {beh} (P{pid})")
                self._play_sound("event")
            return

        key_sym = e.keysym

        matched = []
        for p in self.players:
            beh = p["key_to_behavior"].get(key_sym, None)
            if beh is not None:
                matched.append((p, beh))

        if matched:
            if len(matched) == 1:
                p, beh = matched[0]
                self._handle_behavior_action(p, beh)
            else:
                pid = self._selected_kb_target_player_id()
                p = self._get_player_by_id(pid)
                if p is not None:
                    beh = p["key_to_behavior"].get(key_sym, None)
                    if beh is not None:
                        self._handle_behavior_action(p, beh)
            return

        if key_sym in ("space", "Return"):
            self.toggle_play()
        elif key_sym in ("Left",):
            self.step_frames(-1)
        elif key_sym in ("Right",):
            self.step_frames(1)
        elif key_sym in ("Up",):
            self.step_frames(-100)
        elif key_sym in ("Down",):
            self.step_frames(100)

    def apply_ethogram(self):
        raw = self.eth_var.get().strip()
        behaviors = [b.strip() for b in raw.split(",") if b.strip()]
        if not behaviors:
            behaviors = ["Behaviour1"]
        self.behaviors = behaviors

        for w in self.beh_btn_frame.winfo_children():
            w.destroy()
        for b in self.behaviors:
            btn = ttk.Button(self.beh_btn_frame, text=b, command=lambda bb=b: self.on_behavior_click(bb), style="Ethogram.TButton")
            btn._beh = b

        self._layout_ethogram_buttons()

        self._build_bind_rows()
        self._refresh_active_bout_label()
        self._refresh_bind_labels()
        try:
            self.root.focus_force()
        except Exception:
            pass

    def _layout_ethogram_buttons(self):
        for w in self.beh_btn_frame.winfo_children():
            w.grid_forget()

        btns = list(self.beh_btn_frame.winfo_children())
        if not btns:
            return

        width = max(220, self.beh_scroll_canvas.winfo_width())
        col_w = 200
        cols = max(1, min(4, width // col_w))

        for i in range(cols):
            self.beh_btn_frame.grid_columnconfigure(i, weight=1)

        for idx, btn in enumerate(btns):
            r = idx // cols
            c = idx % cols
            btn.grid(row=r, column=c, sticky="ew", padx=6, pady=4)

        self._on_beh_frame_configure()

    def toggle_play(self):
        if self.cap is None:
            return
        self.playing = not self.playing
        self.btn_play.config(text="Pause" if self.playing else "Play")
        self._last_play_ts = time.time() if self.playing else None
        self._set_overlay("PLAY" if self.playing else "PAUSE", 650)
        self._play_sound("event")

    def step_frames(self, n: int):
        if self.cap is None:
            return
        self.playing = False
        self._last_play_ts = None
        self.btn_play.config(text="Play")
        new_idx = max(0, min(self.frame_count - 1, self.cur_frame_idx + n))
        self.cur_frame_idx = new_idx
        self.slider.set(new_idx)
        self.render_frame(new_idx)

    def on_slider(self, _val):
        if self.cap is None:
            return
        idx = int(float(self.slider.get()))
        if idx != self.cur_frame_idx:
            self.playing = False
            self._last_play_ts = None
            self.btn_play.config(text="Play")
            self.cur_frame_idx = idx
            self.render_frame(idx)

    def _ui_tick(self):
        self._poll_gamepads()

        if self.cap is not None and self.playing:
            now = time.time()
            if self._last_play_ts is None:
                self._last_play_ts = now
            dt = max(0.0, now - self._last_play_ts)
            fps = float(self.fps or 0.0)
            if fps > 0:
                adv = int(dt * fps)
                if adv <= 0:
                    adv = 1
                self._last_play_ts = now
            else:
                adv = 1
                self._last_play_ts = now

            nxt = self.cur_frame_idx + adv
            if nxt >= self.frame_count:
                self.cur_frame_idx = max(0, self.frame_count - 1)
                self.slider.set(self.cur_frame_idx)
                self.render_frame(self.cur_frame_idx)
                self.playing = False
                self._last_play_ts = None
                self.btn_play.config(text="Play")
                self._set_overlay("END OF VIDEO", 900)
                self._play_sound("end")
            else:
                self.cur_frame_idx = nxt
                self.slider.set(nxt)
                self.render_frame(nxt)

        self.root.after(30, self._ui_tick)

    def _pad_btn_pressed(self, player: dict, btn_idx: int) -> bool:
        if not player.get("enabled") or player.get("pad_obj") is None:
            return False
        js = player["pad_obj"]
        try:
            v = int(js.get_button(btn_idx))
        except Exception:
            v = 0

        state = player.get("pad_btn_state", None)
        if state is None:
            try:
                nb = js.get_numbuttons()
            except Exception:
                nb = 0
            state = [0] * max(0, int(nb))
            player["pad_btn_state"] = state

        if btn_idx < 0:
            return False

        if btn_idx >= len(state):
            state.extend([0] * (btn_idx - len(state) + 1))

        prev = state[btn_idx]
        state[btn_idx] = v
        return v == 1 and prev == 0

    def _poll_gamepads(self):
        if not _HAS_PYGAME:
            return
        try:
            pygame.event.pump()
        except Exception:
            return

        if self._binding_player_btn is not None and self._binding_behavior_btn is not None:
            pid = int(self._binding_player_btn)
            beh = str(self._binding_behavior_btn)
            p = self._get_player_by_id(pid)
            if p is not None and p.get("enabled") and p.get("pad_obj") is not None:
                js = p["pad_obj"]
                try:
                    nb = js.get_numbuttons()
                except Exception:
                    nb = 0
                for i in range(nb):
                    if self._pad_btn_pressed(p, i):
                        self._binding_player_btn = None
                        self._binding_behavior_btn = None
                        self._binding_player_key = None
                        self._binding_behavior_key = None
                        self._assign_button(p, beh, i)
                        self.bind_note_var.set(f"Bound button {i} to {beh} (P{pid})")
                        self._play_sound("event")
                        break
            return

        if self.cap is None:
            return

        for p in self.players:
            if p.get("input") != "Gamepad":
                continue
            if not p.get("enabled") or p.get("pad_obj") is None:
                continue
            idx = p.get("pad_idx", None)
            if idx is None:
                continue
            owner = self._pad_owner.get(int(idx), None)
            if owner is not None and int(owner) != int(p.get("id", 0)):
                continue

            js = p["pad_obj"]

            try:
                if js.get_numhats() > 0:
                    hat = js.get_hat(0)
                else:
                    hat = (0, 0)
            except Exception:
                hat = (0, 0)

            if hat != p["pad_last_hat"]:
                p["pad_last_hat"] = hat
                hx, hy = hat
                if hx == -1:
                    self.step_frames(-1)
                elif hx == 1:
                    self.step_frames(1)
                if hy == 1:
                    self.step_frames(-100)
                elif hy == -1:
                    self.step_frames(100)

            now = time.time()
            try:
                ax0 = float(js.get_axis(0)) if js.get_numaxes() > 0 else 0.0
            except Exception:
                ax0 = 0.0

            if abs(ax0) > 0.75 and now - p["pad_repeat_t"] > 0.08:
                p["pad_repeat_t"] = now
                self.step_frames(1 if ax0 > 0 else -1)

            if self._pad_btn_pressed(p, 7):
                self.toggle_play()
            if self._pad_btn_pressed(p, 6):
                self.undo_last()

            for btn_idx, beh in list((p.get("button_to_behavior") or {}).items()):
                if self._pad_btn_pressed(p, int(btn_idx)):
                    self._handle_behavior_action(p, beh)
                    break

    def _draw_hud(self, frame_bgr):
        h, w = frame_bgr.shape[:2]
        scale = max(0.6, min(1.2, w / 1280.0))
        thick = max(1, int(2 * scale))
        pad = int(14 * scale)

        lines = []
        if self.active_bouts:
            by_player = {}
            for (pid, aid, beh), sf in self.active_bouts.items():
                by_player.setdefault(pid, []).append((aid, beh, sf))
            pkeys = sorted(by_player.keys())
            show = []
            for pid in pkeys[:3]:
                items = by_player[pid]
                items_s = []
                for (aid, beh, _sf) in items[:4]:
                    items_s.append(f"A{aid}:{beh}")
                tail = ""
                if len(items) > 4:
                    tail = f" +{len(items) - 4}"
                show.append(f"P{pid}[" + ", ".join(items_s) + f"{tail}]")
            if len(pkeys) > 3:
                show.append(f"+{len(pkeys)-3} players")
            lines.append("ACTIVE: " + " | ".join(show))
        else:
            lines.append("ACTIVE: none")

        mode = self.mode_var.get()
        pol = self.bout_policy_var.get()
        lines.append(f"MODE: {mode}   POLICY: {pol}   PLAYERS: {self.player_count}   ANIMALS: {self.animal_count}")

        y = pad + int(12 * scale)
        for i, s in enumerate(lines):
            cv2.putText(frame_bgr, s, (pad, y + i * int(26 * scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.7 * scale, (0, 0, 0), thick + 2, cv2.LINE_AA)
            cv2.putText(frame_bgr, s, (pad, y + i * int(26 * scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.7 * scale, (245, 245, 245), thick, cv2.LINE_AA)

        if self.overlay_text and time.time() <= self.overlay_until:
            msg = self.overlay_text
            msg_scale = 1.15 * scale
            (tw, th), _ = cv2.getTextSize(msg, cv2.FONT_HERSHEY_SIMPLEX, msg_scale, thick + 1)
            x = max(pad, (w - tw) // 2)
            y0 = max(pad + th + 5, int(h * 0.10))
            cv2.putText(frame_bgr, msg, (x, y0), cv2.FONT_HERSHEY_SIMPLEX, msg_scale, (0, 0, 0), thick + 4, cv2.LINE_AA)
            cv2.putText(frame_bgr, msg, (x, y0), cv2.FONT_HERSHEY_SIMPLEX, msg_scale, (80, 220, 120), thick + 1, cv2.LINE_AA)

        return frame_bgr

    def render_frame(self, idx: int):
        if self.cap is None:
            return
        if self._last_rendered_idx == idx:
            self._update_info_label()
            return

        self.cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = self.cap.read()
        if not ok or frame is None:
            self._update_info_label()
            return

        self._last_rendered_idx = idx

        frame = self._draw_hud(frame)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        canvas_w = max(1, self.canvas.winfo_width())
        canvas_h = max(1, self.canvas.winfo_height())

        h, w = frame_rgb.shape[:2]
        scale = min(canvas_w / w, canvas_h / h)
        new_w = max(1, int(w * scale))
        new_h = max(1, int(h * scale))

        img = Image.fromarray(frame_rgb).resize((new_w, new_h), Image.Resampling.LANCZOS)
        self._tk_img = ImageTk.PhotoImage(img)

        self.canvas.delete("all")
        x0 = (canvas_w - new_w) // 2
        y0 = (canvas_h - new_h) // 2
        self.canvas.create_image(x0, y0, anchor="nw", image=self._tk_img)

        self._update_info_label()

    def _update_info_label(self):
        if self.cap is None or self.frame_count <= 0:
            self.info_var.set("Frame: - / - | Time: - s | Timestamp: -")
            return

        t_s = self._frame_to_seconds(self.cur_frame_idx)
        ts = "-"
        if self.start_dt is not None and self.fps and self.fps > 0:
            ts_dt = self.start_dt + timedelta(seconds=t_s)
            ts = ts_dt.isoformat(sep=" ")

        self.info_var.set(
            f"Frame: {self.cur_frame_idx} / {self.frame_count - 1} | "
            f"Time: {t_s:.3f} s | Timestamp: {ts}"
        )

    def _frame_to_seconds(self, frame_idx: int) -> float:
        if self.fps and self.fps > 0:
            return frame_idx / self.fps
        return float(frame_idx)

    def _frame_to_datetime_str(self, frame_idx: int) -> str:
        if self.start_dt is None or not (self.fps and self.fps > 0):
            return ""
        dt = self.start_dt + timedelta(seconds=self._frame_to_seconds(frame_idx))
        return dt.isoformat(sep=" ")

    def _resolve_action_animal_id(self, player: dict, override_animal: int = None) -> int:
        if override_animal is not None:
            aid = int(override_animal)
        else:
            pa = int(player.get("animal_id", 1) or 1)
            if pa == 0:
                aid = int(self._selected_gui_animal_id())
            else:
                aid = pa
        if aid < 1:
            aid = 1
        if aid > self.animal_count:
            aid = self.animal_count
        return aid

    def on_behavior_click(self, behavior: str):
        if self.cap is None:
            messagebox.showwarning("No video", "Load a video first.")
            self._play_sound("warn")
            return
        pid = self._selected_gui_player_id()
        p = self._get_player_by_id(pid)
        if p is None:
            return
        override_aid = None
        if int(p.get("animal_id", 1) or 1) == 0:
            override_aid = self._selected_gui_animal_id()
        self._handle_behavior_action(p, behavior, override_animal=override_aid)
        try:
            self.root.focus_force()
        except Exception:
            pass

    def _player_allows_behavior(self, player: dict, behavior: str) -> bool:
        allowed = player.get("allowed_behaviors", None)
        if allowed is None:
            return True
        return behavior in allowed

    def _handle_behavior_action(self, player: dict, behavior: str, override_animal: int = None):
        if not self._player_allows_behavior(player, behavior):
            self._set_overlay(f"BLOCKED: {player['name']}", 900)
            self._play_sound("warn")
            return

        mode = self.mode_var.get()
        pid = int(player["id"])
        aid = self._resolve_action_animal_id(player, override_animal=override_animal)

        if mode == "event":
            self._add_event(player, aid, behavior)
            self._set_overlay(f"P{pid} A{aid} EVENT: {behavior}", 900)
            self._play_sound("event")
            self._last_rendered_idx = None
            self.render_frame(self.cur_frame_idx)
            return

        policy = self.bout_policy_var.get()
        key = (pid, aid, behavior)

        if policy == "parallel":
            if key in self.active_bouts:
                sf = int(self.active_bouts.get(key, self.cur_frame_idx))
                dur = self._frame_to_seconds(self.cur_frame_idx) - self._frame_to_seconds(sf)
                self._end_bout(player, aid, behavior, self.cur_frame_idx)
                self._set_overlay(f"P{pid} A{aid} END: {behavior} ({dur:.2f}s)", 1100)
                self._play_sound("end")
            else:
                self.active_bouts[key] = int(self.cur_frame_idx)
                self._set_overlay(f"P{pid} A{aid} START: {behavior}", 900)
                self._play_sound("start")
        else:
            if key in self.active_bouts:
                sf = int(self.active_bouts.get(key, self.cur_frame_idx))
                dur = self._frame_to_seconds(self.cur_frame_idx) - self._frame_to_seconds(sf)
                self._end_bout(player, aid, behavior, self.cur_frame_idx)
                self._set_overlay(f"P{pid} A{aid} END: {behavior} ({dur:.2f}s)", 1100)
                self._play_sound("end")
            else:
                to_end = []
                for (pp, aa, bb), _sf in list(self.active_bouts.items()):
                    if pp == pid and aa == aid:
                        to_end.append(bb)
                ended_any = False
                for bb in to_end:
                    self._end_bout(player, aid, bb, self.cur_frame_idx)
                    ended_any = True
                self.active_bouts[(pid, aid, behavior)] = int(self.cur_frame_idx)
                self._set_overlay(f"P{pid} A{aid} START: {behavior}" + (" (auto-ended)" if ended_any else ""), 1100)
                self._play_sound("start")

        self._refresh_active_bout_label()
        self._last_rendered_idx = None
        self.render_frame(self.cur_frame_idx)

    def _add_event(self, player: dict, animal_id: int, behavior: str):
        sf = self.cur_frame_idx
        ef = self.cur_frame_idx
        st = self._frame_to_seconds(sf)
        et = self._frame_to_seconds(ef)
        sd = self._frame_to_datetime_str(sf)
        ed = self._frame_to_datetime_str(ef)

        ann = Annotation(
            video_path=self.video_path,
            behavior=behavior,
            kind="event",
            start_frame=sf,
            end_frame=ef,
            start_time_s=st,
            end_time_s=et,
            start_datetime=sd,
            end_datetime=ed,
            player_id=int(player["id"]),
            player_name=str(player["name"]),
            animal_id=int(animal_id),
        )
        self.annotations.append(ann)
        self._tree_add(ann)

    def _end_bout(self, player: dict, animal_id: int, behavior: str, end_frame: int):
        key = (int(player["id"]), int(animal_id), behavior)
        if key not in self.active_bouts:
            return

        sf = int(self.active_bouts.get(key, end_frame))
        ef = int(end_frame)
        if ef < sf:
            ef = sf

        st = self._frame_to_seconds(sf)
        et = self._frame_to_seconds(ef)
        sd = self._frame_to_datetime_str(sf)
        ed = self._frame_to_datetime_str(ef)

        ann = Annotation(
            video_path=self.video_path,
            behavior=behavior,
            kind="bout",
            start_frame=sf,
            end_frame=ef,
            start_time_s=st,
            end_time_s=et,
            start_datetime=sd,
            end_datetime=ed,
            player_id=int(player["id"]),
            player_name=str(player["name"]),
            animal_id=int(animal_id),
        )

        self.active_bouts.pop(key, None)
        self.annotations.append(ann)
        self._tree_add(ann)

    def _refresh_active_bout_label(self):
        if not self.active_bouts:
            self.active_bout_var.set("Active bouts: none")
            return
        parts = []
        for (pid, aid, beh), sf in list(self.active_bouts.items())[:18]:
            parts.append(f"P{pid} A{aid} {beh}@{sf}")
        tail = ""
        if len(self.active_bouts) > 18:
            tail = f" (+{len(self.active_bouts)-18})"
        self.active_bout_var.set("Active bouts: " + " | ".join(parts) + tail)

    def _tree_add(self, ann: Annotation):
        vals = (
            f"P{ann.player_id}",
            f"A{ann.animal_id}",
            ann.behavior,
            ann.kind,
            ann.start_frame,
            ann.end_frame,
            f"{ann.start_time_s:.6f}",
            f"{ann.end_time_s:.6f}",
            ann.start_datetime,
            ann.end_datetime,
        )
        self.tree.insert("", tk.END, values=vals)

    def undo_last(self):
        if not self.annotations:
            return
        self.annotations.pop()
        children = self.tree.get_children()
        if children:
            self.tree.delete(children[-1])
        self._set_overlay("UNDO", 700)
        self._play_sound("undo")
        if self.cap is not None:
            self._last_rendered_idx = None
            self.render_frame(self.cur_frame_idx)

    def delete_selected(self):
        sel = self.tree.selection()
        if not sel:
            return
        for item in sel:
            self.tree.delete(item)
        self._rebuild_annotations_from_tree()
        self._set_overlay("DELETED", 700)
        self._play_sound("undo")
        if self.cap is not None:
            self._last_rendered_idx = None
            self.render_frame(self.cur_frame_idx)

    def clear_all(self):
        if not self.annotations and not self.tree.get_children():
            return
        if messagebox.askyesno("Clear all", "Delete ALL saved annotations?"):
            self.annotations = []
            self.active_bouts = {}
            for item in self.tree.get_children():
                self.tree.delete(item)
            self._refresh_active_bout_label()
            self._set_overlay("CLEARED", 700)
            self._play_sound("undo")
            if self.cap is not None:
                self._last_rendered_idx = None
                self.render_frame(self.cur_frame_idx)

    def _rebuild_annotations_from_tree(self):
        anns = []
        for item in self.tree.get_children():
            v = self.tree.item(item, "values")
            player_s = v[0]
            animal_s = v[1]
            try:
                pid = int(str(player_s).replace("P", ""))
            except Exception:
                pid = 1
            try:
                aid = int(str(animal_s).replace("A", ""))
            except Exception:
                aid = 1
            behavior = v[2]
            kind = v[3]
            sf = int(v[4])
            ef = int(v[5])
            st = float(v[6])
            et = float(v[7])
            sd = v[8]
            ed = v[9]
            pname = ""
            p = self._get_player_by_id(pid)
            if p is not None:
                pname = str(p.get("name", ""))
            anns.append(
                Annotation(
                    video_path=self.video_path,
                    behavior=behavior,
                    kind=kind,
                    start_frame=sf,
                    end_frame=ef,
                    start_time_s=st,
                    end_time_s=et,
                    start_datetime=sd,
                    end_datetime=ed,
                    player_id=pid,
                    player_name=pname,
                    animal_id=aid,
                )
            )
        self.annotations = anns

    def _end_all_active_bouts_for_save(self):
        if not self.active_bouts:
            return
        for (pid, aid, beh) in list(self.active_bouts.keys()):
            p = self._get_player_by_id(pid)
            if p is None:
                continue
            self._end_bout(p, aid, beh, self.cur_frame_idx)
        self._refresh_active_bout_label()

    def save_csv(self):
        if not self.annotations and not self.active_bouts:
            messagebox.showwarning("Nothing to save", "No annotations yet.")
            self._play_sound("warn")
            return

        if self.active_bouts:
            if messagebox.askyesno("Active bouts", "You have active bouts. End ALL at the current frame before saving?"):
                self._end_all_active_bouts_for_save()
                self._set_overlay("ENDED ALL ACTIVE", 900)
                self._play_sound("end")

        if not self.annotations:
            messagebox.showwarning("Nothing to save", "No annotations to save.")
            self._play_sound("warn")
            return

        base = "annotations"
        if self.video_path:
            base = os.path.splitext(os.path.basename(self.video_path))[0] + "_annotations"

        out_path = filedialog.asksaveasfilename(
            title="Save results",
            defaultextension=".csv",
            initialfile=f"{base}.csv",
            filetypes=[("CSV", "*.csv"), ("Excel", "*.xlsx")],
        )
        if not out_path:
            return

        rows = []
        for a in self.annotations:
            r = asdict(a)
            r["fps_used"] = self.fps
            r["frame_count"] = self.frame_count
            r["video_filename"] = os.path.basename(self.video_path)
            r["start_datetime_input"] = "" if self.start_dt is None else self.start_dt.isoformat(sep=" ")
            r["mode"] = self.mode_var.get()
            r["bout_policy"] = self.bout_policy_var.get()
            r["players_configured"] = self.player_count
            r["animals_configured"] = self.animal_count
            p = self._get_player_by_id(int(a.player_id))
            if p is not None:
                r["player_input"] = str(p.get("input", ""))
                r["player_pad_name"] = str(p.get("pad_name", ""))
                r["player_pad_guid"] = str(p.get("pad_guid", ""))
                ab = p.get("allowed_behaviors", None)
                r["player_allowed_behaviors"] = "" if ab is None else ",".join(sorted(list(ab)))
                r["player_config_animal_id"] = int(p.get("animal_id", 1) or 1)
            else:
                r["player_input"] = ""
                r["player_pad_name"] = ""
                r["player_pad_guid"] = ""
                r["player_allowed_behaviors"] = ""
                r["player_config_animal_id"] = 0
            rows.append(r)

        df = pd.DataFrame(rows)

        ext = os.path.splitext(out_path)[1].lower().strip()
        if ext == ".xlsx":
            with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
                df.to_excel(writer, index=False, sheet_name="ALL")
                for pid in sorted(df["player_id"].unique().tolist()):
                    dfp = df[df["player_id"] == pid].copy()
                    sheet = f"P{int(pid)}"
                    dfp.to_excel(writer, index=False, sheet_name=sheet[:31])
            messagebox.showinfo("Saved", f"Saved:\n{out_path}")
            self._set_overlay("SAVED XLSX", 900)
            self._play_sound("event")
            return

        if self.player_count <= 1:
            df.to_csv(out_path, index=False)
            messagebox.showinfo("Saved", f"Saved:\n{out_path}")
            self._set_overlay("SAVED CSV", 900)
            self._play_sound("event")
            return

        root_dir = os.path.dirname(out_path) or "."
        base_name = os.path.splitext(os.path.basename(out_path))[0]
        all_path = os.path.join(root_dir, f"{base_name}_ALL.csv")
        df.to_csv(all_path, index=False)
        for pid in sorted(df["player_id"].unique().tolist()):
            dfp = df[df["player_id"] == pid].copy()
            p_path = os.path.join(root_dir, f"{base_name}_P{int(pid)}.csv")
            dfp.to_csv(p_path, index=False)

        messagebox.showinfo("Saved", f"Saved:\n{all_path}\n(and per-player CSVs in same folder)")
        self._set_overlay("SAVED CSVs", 900)
        self._play_sound("event")

    def show_controls_popup(self):
        win = tk.Toplevel(self.root)
        win.title("Controls / Mapping")
        win.geometry("820x560")
        win.configure(bg="#111316")

        box = ttk.Frame(win)
        box.pack(fill=tk.BOTH, expand=True, padx=12, pady=12)

        txt = tk.Text(box, wrap="word", bg="#0E1013", fg="#EAEAEA", insertbackground="#EAEAEA")
        txt.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        sb = ttk.Scrollbar(box, orient="vertical", command=txt.yview)
        sb.pack(side=tk.RIGHT, fill=tk.Y)
        txt.configure(yscrollcommand=sb.set)

        lines = []
        lines.append("VIDEO CONTROLS")
        lines.append("  Keyboard: Space/Enter = Play/Pause")
        lines.append("            ← / →      = -1 / +1 frame")
        lines.append("            ↑ / ↓      = -100 / +100 frames")
        lines.append("")
        lines.append("  Gamepad:")
        lines.append("            D-pad left/right = -1 / +1 frame")
        lines.append("            D-pad up/down    = -100 / +100 frames")
        lines.append("            Left stick X     = continuous seek")
        lines.append("            Button 7 (Start) = Play/Pause")
        lines.append("            Button 6 (Back)  = Undo last annotation")
        lines.append("")
        lines.append("MULTIPLAYER / MULTI-ANIMAL")
        lines.append("  Ethogram buttons use GUI Player.")
        lines.append("  GUI Animal is only used when that GUI Player is set to Animal=ALL in Players/Animals.")
        lines.append("  Keyboard behaviour keys: if multiple players share a key, Keyboard targets player selector decides.")
        lines.append("  Each physical gamepad can be assigned to only ONE player; duplicates are disabled.")
        lines.append("")
        txt.insert("1.0", "\n".join(lines))
        txt.configure(state="disabled")

    def open_hyperlink(self, url: str):
        import webbrowser
        webbrowser.open(url)

    def load_video(self):
        path = filedialog.askopenfilename(
            title="Select video",
            filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv *.m4v"), ("All files", "*.*")],
        )
        if not path:
            return

        self._close_video()

        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            messagebox.showerror("Error", "Could not open video.")
            return

        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps is None or fps <= 0 or math.isnan(fps):
            fps = 0.0

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)

        self.cap = cap
        self.video_path = path
        self.fps = float(fps)
        self.frame_count = frame_count
        self.cur_frame_idx = 0
        self.playing = False
        self._last_play_ts = None
        self._last_rendered_idx = None

        self.video_label.config(text=os.path.basename(path))
        self.fps_var.set("" if self.fps <= 0 else f"{self.fps:.3f}")

        self.slider.configure(from_=0, to=max(0, self.frame_count - 1))
        self.slider.set(0)

        self.btn_play.config(state=tk.NORMAL, text="Play")

        self._set_overlay("VIDEO LOADED", 900)
        self._play_sound("event")
        self.render_frame(0)
        try:
            self.root.focus_force()
        except Exception:
            pass

    def _close_video(self):
        if self.cap is not None:
            try:
                self.cap.release()
            except Exception:
                pass
        self.cap = None
        self.video_path = ""
        self.frame_count = 0
        self.cur_frame_idx = 0
        self.playing = False
        self._last_play_ts = None
        self._last_rendered_idx = None
        try:
            self.canvas.delete("all")
        except Exception:
            pass
        try:
            self.btn_play.config(state=tk.DISABLED, text="Play")
        except Exception:
            pass
        try:
            self.slider.configure(from_=0, to=0)
            self.slider.set(0)
        except Exception:
            pass
        try:
            self.video_label.config(text="No video loaded")
        except Exception:
            pass
        self._update_info_label()


def main():
    root = tk.Tk()
    app = BehaviourAnnotatorGUI(root)
    root.mainloop()


if __name__ == "__main__":
    main()
